In [1]:
import torch
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import cv2
import torchvision
from torchvision import transforms
import albumentations
import albumentations.pytorch

In [35]:
torch.manual_seed(777)
device = torch.device("cuda")
torch.cuda.manual_seed_all(777)

In [21]:
# Dataset
class Dataset(Dataset):
    def __init__(self, transform = None):
        super().__init__()
        
        self.img_dir = "/home/temp_1/kangsanha/malocclusion/data"
        self.filenames = os.listdir(self.img_dir)

        self.transform = transform

    def __getitem__(self, index):
        label = MakeLabel(self.filenames[index])
        img_path = os.path.join(self.img_dir, self.filenames[index])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB
        # albumentations 라이브러리를 사용할 것인데, PIL을 쓰면 변환이 안된다하여 cv2로 변환해준다.
        # img = np.asarray_chkfinite(Image.open(img_path))
        
        if img.dtype == np.uint8:
            img = img / 255.0
        
        #img = ToTensor(img)
        #print(type(img))
        #print(np.shape(img))
        #img = img.transpose((2,0,1)).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img)
            img = aug['image']
        #img = img.values()
        
        #  print(np.shape(img)) # 검사코드
        # input = {'data' : img, 'label' : label}
        
        return img, label
        

    def __len__(self):
        
        return len(self.filenames)

    
def MakeLabel(filenames):
    input_img = filenames
    if "O" in input_img:
        label = 1#label = F.one_hot(torch.tensor(0), 2)
    else : label = 0#label = F.one_hot(torch.tensor(1), 2)

    return label

def ToTensor(image):
    image = image.transpose((2,0,1)).astype(np.float32)
    image = torch.from_numpy(image)
    return image

In [22]:
# albumentations train

albumentations_train = albumentations.Compose([
    albumentations.Resize(height=256, width=256),
    albumentations.OneOf([
        albumentations.HorizontalFlip(p=0.8),
        albumentations.RandomRotate90(p=0.8),
        albumentations.VerticalFlip(p=0.8)
    ], p=1),
    albumentations.OneOf([
        albumentations.MotionBlur(p=0.75),
        albumentations.OpticalDistortion(p=0.75),
        albumentations.GaussNoise(p=0.75)
    ], p=1),
    albumentations.pytorch.ToTensorV2()
])

In [23]:
full_dataset = Dataset(transform = albumentations_train)

In [24]:
full_dataset[0]

(tensor([[[0.7350, 0.7356, 0.7378,  ..., 0.6962, 0.7059, 0.7044],
          [0.7333, 0.7349, 0.7351,  ..., 0.7193, 0.7214, 0.7163],
          [0.7323, 0.7374, 0.7360,  ..., 0.7145, 0.7226, 0.7243],
          ...,
          [0.3920, 0.3892, 0.3900,  ..., 0.6817, 0.6747, 0.6775],
          [0.3936, 0.4000, 0.3982,  ..., 0.6845, 0.6805, 0.6875],
          [0.3980, 0.3903, 0.3940,  ..., 0.6908, 0.6975, 0.6938]],
 
         [[0.5225, 0.5231, 0.5279,  ..., 0.4811, 0.4928, 0.4952],
          [0.5236, 0.5256, 0.5243,  ..., 0.5043, 0.5065, 0.5004],
          [0.5249, 0.5282, 0.5325,  ..., 0.4908, 0.5054, 0.5062],
          ...,
          [0.2538, 0.2510, 0.2594,  ..., 0.5452, 0.5364, 0.5425],
          [0.2581, 0.2643, 0.2653,  ..., 0.5436, 0.5367, 0.5533],
          [0.2677, 0.2605, 0.2603,  ..., 0.5539, 0.5613, 0.5629]],
 
         [[0.4134, 0.4156, 0.4204,  ..., 0.3671, 0.3735, 0.3776],
          [0.4124, 0.4185, 0.4168,  ..., 0.3898, 0.3901, 0.3818],
          [0.4137, 0.4168, 0.4204,  ...,

In [25]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_datset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [26]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_datset, batch_size=4, shuffle=False, num_workers=0)

In [27]:
# model(ResNet50)

import torchvision.models.resnet as resnet
import torch.optim as optim

conv1x1 = resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock = resnet.BasicBlock

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2, zero_init_residual=True):
        super(ResNet, self).__init__()
        self.inplanes = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False) # 마찬가지로 전부 사이즈 조정
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 32, layers[0], stride=1) # 3 반복
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2) # 4 반복
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2) # 6 반복
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2) # 3 반복
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1): # planes -> 입력되는 채널 수
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion: 
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input [32, 128, 128] -> [C ,H, W]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #x.shape =[32, 64, 64]

        x = self.layer1(x)
        #x.shape =[128, 64, 64]
        x = self.layer2(x)
        #x.shape =[256, 32, 32]
        x = self.layer3(x)
        #x.shape =[512, 16, 16]
        x = self.layer4(x)
        #x.shape =[1024, 8, 8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [28]:
resnet50 = ResNet(resnet.Bottleneck, [3,4,6,3], 2, True)

In [29]:
resnet50.cuda()

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 

In [30]:
# 정의
optimizer = torch.optim.Adam(resnet50.parameters(), lr=1e-5)
loss_func = nn.CrossEntropyLoss().to(device)

In [31]:
from torchsummary import summary
summary(resnet50, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
         MaxPool2d-4           [-1, 32, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]           1,024
       BatchNorm2d-6           [-1, 32, 64, 64]              64
              ReLU-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 32, 64, 64]           9,216
       BatchNorm2d-9           [-1, 32, 64, 64]              64
             ReLU-10           [-1, 32, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]           4,096
      BatchNorm2d-12          [-1, 128, 64, 64]             256
           Conv2d-13          [-1, 128, 64, 64]           4,096
      BatchNorm2d-14          [-1, 128,

In [32]:
mode = 'train'

In [36]:
# train
if mode == 'train':
    for epoch in range(0 , 100):
        resnet50.train()
        loss_arr = []
        for batch, data in enumerate(train_loader, 1):
            inputs, labels = data # input data, label 분리
            inputs = inputs.to(device).float()
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = resnet50(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            
            loss_arr += [loss.item()]
            
         
        print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                  (epoch, 100, batch, 135, np.mean(loss_arr)))

TRAIN: EPOCH 0000 / 0100 | BATCH 0135 / 0135 | LOSS 0.6648
TRAIN: EPOCH 0001 / 0100 | BATCH 0135 / 0135 | LOSS 0.6624
TRAIN: EPOCH 0002 / 0100 | BATCH 0135 / 0135 | LOSS 0.6605
TRAIN: EPOCH 0003 / 0100 | BATCH 0135 / 0135 | LOSS 0.6607
TRAIN: EPOCH 0004 / 0100 | BATCH 0135 / 0135 | LOSS 0.6578
TRAIN: EPOCH 0005 / 0100 | BATCH 0135 / 0135 | LOSS 0.6576
TRAIN: EPOCH 0006 / 0100 | BATCH 0135 / 0135 | LOSS 0.6550
TRAIN: EPOCH 0007 / 0100 | BATCH 0135 / 0135 | LOSS 0.6504
TRAIN: EPOCH 0008 / 0100 | BATCH 0135 / 0135 | LOSS 0.6452
TRAIN: EPOCH 0009 / 0100 | BATCH 0135 / 0135 | LOSS 0.6529
TRAIN: EPOCH 0010 / 0100 | BATCH 0135 / 0135 | LOSS 0.6504
TRAIN: EPOCH 0011 / 0100 | BATCH 0135 / 0135 | LOSS 0.6522
TRAIN: EPOCH 0012 / 0100 | BATCH 0135 / 0135 | LOSS 0.6429
TRAIN: EPOCH 0013 / 0100 | BATCH 0135 / 0135 | LOSS 0.6488
TRAIN: EPOCH 0014 / 0100 | BATCH 0135 / 0135 | LOSS 0.6576
TRAIN: EPOCH 0015 / 0100 | BATCH 0135 / 0135 | LOSS 0.6458
TRAIN: EPOCH 0016 / 0100 | BATCH 0135 / 0135 | LOSS 0.64

KeyboardInterrupt: 